In [16]:
import numpy as np
import pandas as pd

In [17]:
data = pd.read_csv('data.csv', index_col=0)
# data.fillna(v÷alue='', inplace = True)
# data = data.replace(to_replace=np.nan(), value="hello").dropna()
# data['date'] = pd.to_datetime(data['date']).to_timestamp

# labels = data[['type']]
# data.drop(['type', 'date', 'userId', 'lat', 'lon', 'location', 'device'],axis=1, inplace=True)
# data.isnull()

In [18]:
#round data
def round(df):
    return df.apply(lambda x: np.rint(x))

data['temp'] = round(data['temp'])
data['windDeg'] = round(data['windDeg'])
data['windSpeed'] = round(data['windSpeed'])

In [19]:
# data.drop(['type', 'date'],axis=1, inplace=True)
data['clouds'].replace(5, 1,inplace=True)
data['clouds'].replace(20, 40,inplace=True)
# data.head()

In [20]:
#one- hot encoding
cat_vars=['weatherCond']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
data = data.drop(['weatherCond'], axis=1)
data

,clouds,humidity,pressure,temp,visibility,windDeg,windSpeed,zipcode,weatherCond_Clear,weatherCond_Clouds,...,weatherCond_Light Freezing Rain,weatherCond_Light Rain,weatherCond_Light Snow,weatherCond_Mist,weatherCond_Mostly Cloudy,weatherCond_None,weatherCond_Overcast,weatherCond_Partly Cloudy,weatherCond_Rain,weatherCond_Scattered Clouds
01125B66-EB0B-455F-8672-48E6413510A5,1,48,1020,40.0,16093,300.0,17.0,10923,0,0,...,0,0,0,0,0,0,0,1,0,0
01EAB7E7-D5EB-407A-A307-CC8CEF6AB7F0,90,82,1015,65.0,16093,130.0,6.0,7310,0,0,...,0,0,0,0,0,0,1,0,0,0
0230E916-2DDC-4721-AC51-441FD7A60F39,90,100,1014,56.0,4828,117.0,3.0,10923,0,0,...,0,0,0,0,0,0,1,0,0,0
03567C18-EC38-4A51-AD35-65D8ADACF0EF,90,87,1022,55.0,16093,0.0,4.0,10002,0,0,...,0,0,0,0,0,0,1,0,0,0
03817C89-1E06-492A-A5ED-AFDE8612C06E,40,62,1029,21.0,32186,260.0,13.0,10956,1,0,...,0,0,0,0,0,0,0,0,0,0
03B00035-A25A-4590-94F7-090CC9E614D9,1,75,1016,43.0,16093,258.0,9.0,10705,0,0,...,0,0,0,0,0,0,0,0,0,1
04b593d0-5dd7-11e7-8932-b96748e46eb9,75,66,1012,87.0,24140,250.0,6.0,10927,0,0,...,0,0,0,0,1,0,0,0,0,0
04e01fe0-5dcf-11e7-a931-a7582745470f,40,49,1013,89.0,16093,210.0,16.0,10927,0,0,...,0,0,0,0,1,0,0,0,0,0
066774CF-9E45-46A8-8173-A8D415A38931,90,81,1028,40.0,9656,0.0,4.0,10002,0,0,...,0,0,0,0,0,0,1,0,0,0
0689E459-C7F3-4ED7-8582-58FF1C10B3D4,75,55,1014,61.0,16093,120.0,5.0,10901,0,0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
# categorize values
data = data.fillna({"visibility": 0})
data = data.fillna({"windSpeed": 0})
for index,rows in data.iterrows():
    if (data.loc[index, 'visibility'] >0 and data.loc[index, 'visibility'] < 10000):
        data.loc[index, 'visibility'] = 1
    elif (data.loc[index, 'visibility'] > 10000 and data.loc[index, 'visibility'] < 20000):
        data.loc[index, 'visibility'] = 2
    elif (data.loc[index, 'visibility'] > 20000 and data.loc[index, 'visibility'] < 30000):
        data.loc[index, 'visibility'] = 3
    else:
        data.loc[index, 'visibility'] = 4
        
for index,rows in data.iterrows():
    if (data.loc[index, 'temp'] >0 and data.loc[index, 'temp'] < 32):
        data.loc[index, 'temp'] = 1
    elif (data.loc[index, 'temp'] > 32 and data.loc[index, 'temp'] < 60):
        data.loc[index, 'temp'] = 2
    elif (data.loc[index, 'temp'] > 60 and data.loc[index, 'temp'] < 90):
        data.loc[index, 'temp'] = 3
    else:
        data.loc[index, 'temp'] = 4
        
for index,rows in data.iterrows():
    if (data.loc[index, 'windDeg'] >0 and data.loc[index, 'windDeg'] < 90):
        data.loc[index, 'windDeg'] = 1
    elif (data.loc[index, 'windDeg'] > 90 and data.loc[index, 'windDeg'] < 180):
        data.loc[index, 'windDeg'] = 2
    elif (data.loc[index, 'windDeg'] > 180 and data.loc[index, 'windDeg'] < 270):
        data.loc[index, 'windDeg'] = 3
    else:
        data.loc[index, 'windDeg'] = 4
        
for index,rows in data.iterrows():
    if (data.loc[index, 'windSpeed'] >0 and data.loc[index, 'windSpeed'] < 5):
        data.loc[index, 'windSpeed'] = 1
    elif (data.loc[index, 'windSpeed'] > 5 and data.loc[index, 'windSpeed'] < 10):
        data.loc[index, 'windSpeed'] = 2
    else:
        data.loc[index, 'windSpeed'] = 3

In [22]:
data.head()

,clouds,humidity,pressure,temp,visibility,windDeg,windSpeed,zipcode,weatherCond_Clear,weatherCond_Clouds,...,weatherCond_Light Freezing Rain,weatherCond_Light Rain,weatherCond_Light Snow,weatherCond_Mist,weatherCond_Mostly Cloudy,weatherCond_None,weatherCond_Overcast,weatherCond_Partly Cloudy,weatherCond_Rain,weatherCond_Scattered Clouds
01125B66-EB0B-455F-8672-48E6413510A5,1,48,1020,2.0,2,4.0,3.0,10923,0,0,...,0,0,0,0,0,0,0,1,0,0
01EAB7E7-D5EB-407A-A307-CC8CEF6AB7F0,90,82,1015,3.0,2,2.0,2.0,7310,0,0,...,0,0,0,0,0,0,1,0,0,0
0230E916-2DDC-4721-AC51-441FD7A60F39,90,100,1014,2.0,1,2.0,1.0,10923,0,0,...,0,0,0,0,0,0,1,0,0,0
03567C18-EC38-4A51-AD35-65D8ADACF0EF,90,87,1022,2.0,2,4.0,1.0,10002,0,0,...,0,0,0,0,0,0,1,0,0,0
03817C89-1E06-492A-A5ED-AFDE8612C06E,40,62,1029,1.0,4,3.0,3.0,10956,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
rfe = RFE(model, 4)
rfe = rfe.fit(data, labels)
print(rfe.support_)
print(rfe.ranking_)

[False False False False False False False False  True  True False False
 False False False  True False False  True False False False False False]
[16 17 19 18  9  3  7 21  1  1 12 11 15 14 20  1 13  8  1  5 10  2  6  4]


/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [24]:
X = data#data[['lat','lon', 'temp', 'windSpeed']]
y = labels
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
print('Logistic regression accuracy: {:.3f}'.format(accuracy_score(y_test, logreg.predict(X_test))))

Logistic regression accuracy: 0.978


/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [26]:
from sklearn.metrics import accuracy_score
print('Logistic regression accuracy: {:.3f}'.format(accuracy_score(y_test, logreg.predict(X_test))))

Logistic regression accuracy: 0.978


In [27]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print('Random Forest Accuracy: {:.3f}'.format(accuracy_score(y_test, rf.predict(X_test))))

Random Forest Accuracy: 0.967


/Users/antoniohung/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [28]:
#SVM
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
print('Support vector machine accuracy: {:.3f}'.format(accuracy_score(y_test, svc.predict(X_test))))              

Support vector machine accuracy: 0.978


/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
from sklearn import model_selection as ms
from sklearn.model_selection import cross_val_score
scoring = 'accuracy'


kfoldMean = ms.cross_val_score(svc, X_train, y_train, cv = ms.KFold(), scoring=scoring).mean()
Kfold_shuggle = ms.cross_val_score(svc, X_train, y_train, cv = ms.KFold(shuffle = True), scoring=scoring).mean()
print(kfoldMean)
print(Kfold_shuggle)

0.951690821256
0.946859903382


/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/antoniohung/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expec

In [271]:
#save mode
import pickle
filename = 'svc.sav'
# filename = 'rf.sav'
pickle.dump(svc, open(filename, 'wb'), protocol=2)

In [272]:
file = open(filename, 'rb')
svc = pickle.load(file)
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)